In [142]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_iris
from scipy.stats import chi2_contingency
import re
from datetime import datetime
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

df = pd.read_pickle(r'/Users/maya/Documents/Information Systems/שנה ג׳/למידת מכונה/פרוייקט קורס/XY_train.pkl')
df.head()

,textID,text,sentiment,message_date,account_creation_date,previous_messages_dates,date_of_new_follower,date_of_new_follow,email,gender,email_verified,blue_tick,embedded_content,platform
0,b8f4e560fa,but yeah i like purple maybe thats why!! ;) ...,positive,2023-10-2 7:41:52,2015-4-13 0:1:16,"[2019-2-15 16:12:42, 2020-6-2 16:49:53, 2021-4...","[2016-1-13 7:12:26, 2016-5-17 13:40:38, 2016-1...","[2016-7-15 0:3:50, 2016-4-10 22:31:8, 2016-10-...",ClareRosindill3265@messenger.gov,F,NaN,False,mp4,facebook
1,f81a1511b2,_Henrie haha i WISH i coudl meet you.. you sho...,positive,2022-11-17 2:1:57,2013-7-1 18:58:16,"[2021-7-5 21:53:48, 2020-12-12 9:7:50, 2020-9-...","[2016-10-4 7:49:46, 2016-6-10 7:47:39, 2016-4-...","[2016-7-5 20:23:57, 2016-5-3 15:9:51, 2018-11-...",ErenaAntonchik231@messenger.gov,M,True,True,False,tiktok
2,3e9e3f0d69,"nah, you`re just altered forever Enjoy.",positive,2022-6-1 6:53:18,2013-8-8 2:21:12,"[2021-7-11 9:59:36, 2020-2-18 6:38:45, 2021-5-...","[2018-7-13 17:0:3, 2016-2-13 17:8:12, 2016-2-1...","[2016-7-9 22:15:10, 2018-4-13 20:37:55, 2016-1...",GodardCowlas6687@api.gov,F,True,True,False,tiktok
3,a068b95bd9,Can you ask Ryan why he stopped following me ...,negative,2022-1-4 11:38:57,2015-10-16 14:51:19,"[2021-8-16 4:55:32, 2021-6-1 10:13:37, 2021-12...",[2018-7-1 5:9:39],"[2018-10-18 7:23:20, 2018-4-10 12:17:6, 2016-6...",RalfStolberger6467@python.gov,M,False,False,mp4,tiktok
4,e6da2d1835,New issue of in the office...desperately want...,negative,2022-5-11 21:56:27,2013-12-15 19:8:40,"[2021-9-11 10:55:6, 2021-5-6 12:1:54, 2021-1-1...","[2018-11-2 11:16:12, 2018-10-18 16:8:34]","[2018-4-18 20:56:34, 2018-7-1 14:56:33, 2016-4...",MadelLeaton248@bbc.gov.il,M,False,False,None,None


## Pre-Processing Function

In [143]:
import numpy as np
import pandas as pd
import re
from nltk.stem import SnowballStemmer


def preprocess_data(df):
    # Calculate missing values before deletion
    missing_values_before = df.isnull().sum()

    # Drop rows with missing values exceeding a threshold
    df = df.dropna(thresh=df.shape[1] - 2)
    # 
    # missing_values_after = df.isnull().sum()
    # missing_values_table = pd.DataFrame({
    #     'Before': missing_values_before,
    #     'After': missing_values_after,
    #     'Difference': missing_values_before - missing_values_after
    # })
    # print("Missing Values Before and After Deletion:")
    # print(missing_values_table)

    # Fill missing values for 'email' with 'unknown'
    df['email'] = df['email'].fillna('unknown')

    # Fill missing values for 'embedded_content' and 'platform' based on their probability distributions
    embedded_content_prob = df['embedded_content'].value_counts(normalize=True)
    platform_prob = df['platform'].value_counts(normalize=True)

    def impute_missing_values(row, prob_dist):
        if pd.isnull(row):
            return np.random.choice(prob_dist.index, p=prob_dist.values)
        else:
            return row

    df['embedded_content'] = df['embedded_content'].apply(lambda x: impute_missing_values(x, embedded_content_prob))
    df['platform'] = df['platform'].apply(lambda x: impute_missing_values(x, platform_prob))

    # Fill missing values for 'email_verified' and 'blue_tick'
    df['email_verified'].fillna(df['blue_tick'], inplace=True)
    df['blue_tick'].fillna(df['email_verified'], inplace=True)

    # Fill missing values for 'gender' randomly
    df['gender'].replace('None', np.nan, inplace=True)
    gender_counts = df['gender'].value_counts()
    df['gender'].fillna(pd.Series(np.random.choice(gender_counts.index, size=len(df.index),
                                                   p=(gender_counts / gender_counts.sum()))), inplace=True)

    # Delete the remaining rows with missing values
    df = df.dropna()

    # Convert 'message_date' to categorical and create 'message_time_category' column
    df['message_date'] = pd.to_datetime(df['message_date'])
    df['hour'] = df['message_date'].dt.hour
    morning_interval = range(6, 12)  # 6:00 AM to 11:59 AM
    noon_interval = range(12, 18)  # 12:00 PM to 5:59 PM
    evening_interval = range(18, 24)  # 6:00 PM to 11:59 PM

    def categorize_hour(hour):
        if hour in morning_interval:
            return 'Morning'
        elif hour in noon_interval:
            return 'Noon'
        elif hour in evening_interval:
            return 'Evening'
        else:
            return 'Night'

    df['message_time_category'] = df['hour'].apply(categorize_hour)
    df.drop(columns=['hour'], inplace=True)

    # handle changes for ngram
    # Make a copy of the DataFrame to avoid modifying the original
    df_processed = df.copy()

    # Lowercase
    df_processed['clean_text'] = df_processed['text'].str.lower()

    # Stemming
    stemmer = SnowballStemmer('english')
    df_processed['clean_text'] = df_processed['clean_text'].apply(
        lambda x: ' '.join(stemmer.stem(word) for word in x.split()))

    # Remove punctuation
    df_processed['clean_text'] = df_processed['clean_text'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

    # Remove numbers
    df_processed['clean_text'] = df_processed['clean_text'].apply(
        lambda x: ' '.join(word for word in x.split() if not word.isdigit()))

    # Remove words with 1 letter
    df_processed['clean_text'] = df_processed['clean_text'].apply(lambda x: re.sub(r'\b\w{1}\b', '', x))

    # Remove top 0.05% of most common or not common words
    h_pct = 0.05
    l_pct = 0.05

    # Remove the top $h_pct of the most frequent words
    high_freq = pd.Series(' '.join(df_processed['clean_text']).split()).value_counts()[
                :int(pd.Series(' '.join(df_processed['clean_text']).split()).count() * h_pct / 100)]
    df_processed['clean_text'] = df_processed['clean_text'].apply(
        lambda x: ' '.join(word for word in x.split() if word not in high_freq))

    # Remove the top $l_pct of the least frequent words
    low_freq = pd.Series(' '.join(df_processed['clean_text']).split()).value_counts()[
               :-int(pd.Series(' '.join(df_processed['clean_text']).split()).count() * l_pct / 100):-1]
    df_processed['clean_text'] = df_processed['clean_text'].apply(
        lambda x: ' '.join(word for word in x.split() if word not in low_freq))

    # Remove double spaces
    df_processed['clean_text'] = df_processed['clean_text'].apply(lambda x: re.sub(r'\s+', ' ', x))

    return df_processed


preprocess_test = preprocess_data(df)
preprocess_test.head()


/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_31221/1051864747.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['email'] = df['email'].fillna('unknown')
/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_31221/1051864747.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embedded_content'] = df['embedded_content'].apply(lambda x: impute_missing_values(x, embedded_content_prob))
/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_31221/1051864747.py:37: SettingWithCop

,textID,text,sentiment,message_date,account_creation_date,previous_messages_dates,date_of_new_follower,date_of_new_follow,email,gender,email_verified,blue_tick,embedded_content,platform,message_time_category,clean_text
0,b8f4e560fa,but yeah i like purple maybe thats why!! ;) ...,positive,2023-10-02 07:41:52,2015-4-13 0:1:16,"[2019-2-15 16:12:42, 2020-6-2 16:49:53, 2021-4...","[2016-1-13 7:12:26, 2016-5-17 13:40:38, 2016-1...","[2016-7-15 0:3:50, 2016-4-10 22:31:8, 2016-10-...",ClareRosindill3265@messenger.gov,F,False,False,mp4,facebook,Morning,yeah purpl mayb why
1,f81a1511b2,_Henrie haha i WISH i coudl meet you.. you sho...,positive,2022-11-17 02:01:57,2013-7-1 18:58:16,"[2021-7-5 21:53:48, 2020-12-12 9:7:50, 2020-9-...","[2016-10-4 7:49:46, 2016-6-10 7:47:39, 2016-4-...","[2016-7-5 20:23:57, 2016-5-3 15:9:51, 2018-11-...",ErenaAntonchik231@messenger.gov,M,True,True,False,tiktok,Night,_henri haha wish coudl meet should stop by sea...
2,3e9e3f0d69,"nah, you`re just altered forever Enjoy.",positive,2022-06-01 06:53:18,2013-8-8 2:21:12,"[2021-7-11 9:59:36, 2020-2-18 6:38:45, 2021-5-...","[2018-7-13 17:0:3, 2016-2-13 17:8:12, 2016-2-1...","[2016-7-9 22:15:10, 2018-4-13 20:37:55, 2016-1...",GodardCowlas6687@api.gov,F,True,True,False,tiktok,Morning,nah alter forev enjoy
3,a068b95bd9,Can you ask Ryan why he stopped following me ...,negative,2022-01-04 11:38:57,2015-10-16 14:51:19,"[2021-8-16 4:55:32, 2021-6-1 10:13:37, 2021-12...",[2018-7-1 5:9:39],"[2018-10-18 7:23:20, 2018-4-10 12:17:6, 2016-6...",RalfStolberger6467@python.gov,M,False,False,mp4,tiktok,Morning,ask ryan whi he stop follow twitter
4,e6da2d1835,New issue of in the office...desperately want...,negative,2022-05-11 21:56:27,2013-12-15 19:8:40,"[2021-9-11 10:55:6, 2021-5-6 12:1:54, 2021-1-1...","[2018-11-2 11:16:12, 2018-10-18 16:8:34]","[2018-4-18 20:56:34, 2018-7-1 14:56:33, 2016-4...",MadelLeaton248@bbc.gov.il,M,False,False,False,facebook,Evening,new issu office desper flick through ll wait


## Feature Extraction Function

In [144]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer


def extract_features(df):
    # 1. Create a new column based on the length of messages
    df['message_length'] = df['text'].apply(lambda x: len(x))

    # 2. Create the number of messages sent by the user
    df['num_messages_sent'] = df['previous_messages_dates'].apply(len)

    # 3. Create the number of followers and following
    df['follower_count'] = df['date_of_new_follower'].apply(lambda x: len(x))
    df['following_count'] = df['date_of_new_follow'].apply(lambda x: len(x))

    # 4. Append new columns created to the df
    new_columns_df = df[['follower_count', 'following_count']]

    # 5. N-GRAM
    X = df['clean_text']
    ngram_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=100)
    X_ngrams = ngram_vectorizer.fit_transform(X).toarray()
    df_output = pd.DataFrame(data=X_ngrams, columns=ngram_vectorizer.get_feature_names_out())

    # 6. Extract email domain endings
    def extract_email_domain_ending(email):
        if pd.isnull(email):
            return 'Missing'
        match = re.search(r'\.(\w+)$', email)
        if match:
            return match.group(1)
        else:
            return 'Unknown'

    df['email_domain_ending'] = df['email'].apply(extract_email_domain_ending)
    email_domain_ending_counts = df['email_domain_ending'].value_counts()

    # 7. Create seniority in years
    df['account_creation_date'] = pd.to_datetime(df['account_creation_date'])
    current_date = datetime.now()
    df['seniority'] = (current_date - df['account_creation_date']).dt.days / 365.25

    # 8. Create the average time difference between messages
    def calculate_average_time_difference(message_dates_array):
        message_dates = [datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S') for date_str in message_dates_array]
        time_diffs = []
        for i in range(1, len(message_dates)):
            time_diff = (message_dates[i] - message_dates[i - 1]).total_seconds()
            if time_diff < 0:
                time_diff = (message_dates[i - 1] - message_dates[i]).total_seconds()
            time_diffs.append(time_diff)
        if len(time_diffs) > 0:
            average_time_difference = sum(time_diffs) / len(time_diffs)
            average_time_difference = int(average_time_difference)
            return average_time_difference
        else:
            return None

    df['average_time_difference'] = df['previous_messages_dates'].apply(calculate_average_time_difference)

    return df, new_columns_df, df_output


test_df, new_columns_test, df_test_output = extract_features(preprocess_test)


## Feature Representation Function

In [145]:
from sklearn.preprocessing import MinMaxScaler


def feature_representation(df, new_columns_df, df_output):
    # Normalize the values by dividing each column by its maximum value
    columns_to_normalize = ['message_length', 'num_messages_sent', 'follower_count', 'following_count', 'seniority']
    for column in columns_to_normalize:
        max_value = df[column].max()
        df[f'normalized_{column}'] = df[column] / max_value

    # One-hot coding
    email_domain_ending_onehot = pd.get_dummies(df['email_domain_ending'], prefix='email_ending')
    embedded_content_onehot = pd.get_dummies(df['embedded_content'], prefix='embedded_content')
    platform_onehot = pd.get_dummies(df['platform'], prefix='platform')
    message_time_category_onehot = pd.get_dummies(df['message_time_category'], prefix='message_time')
    df = pd.concat(
        [df, email_domain_ending_onehot, embedded_content_onehot, platform_onehot, message_time_category_onehot],
        axis=1)
    df.drop(['email_domain_ending', 'embedded_content', 'platform', 'message_time_category'], axis=1, inplace=True)

    # Convert one-hot coding to binary values
    bool_to_binary = {True: 1, False: 0}
    df['email_verified'] = df['email_verified'].map(bool_to_binary)
    df['blue_tick'] = df['blue_tick'].map(bool_to_binary)
    for column in df.columns:
        if df[column].dtype == bool:
            df[column] = df[column].astype(int)

    gender_to_binary = {'F': 1, 'M': 0}
    df['gender'] = df['gender'].map(gender_to_binary)

    # Normalization for the n-gram features
    scaler = MinMaxScaler()
    X_ngrams_normalized = scaler.fit_transform(df_output)
    df_normalized = pd.DataFrame(X_ngrams_normalized, columns=df_output.columns)
    df = pd.concat([df.reset_index(drop=True), df_normalized], axis=1)

    # Arrange data again 
    sentiment_mapping = {'positive': 1, 'negative': -1}
    df['sentiment'] = df['sentiment'].map(sentiment_mapping)
    df = df.drop(columns=['text', 'previous_messages_dates', 'message_date',
                          'email', 'date_of_new_follower', 'date_of_new_follow',
                          'account_creation_date', 'message_length', 'num_messages_sent',
                          'follower_count', 'following_count',
                          'seniority', 'clean_text', 'average_time_difference'])

    return df


feature_representation_test = feature_representation(test_df, new_columns_test, df_test_output)
feature_representation_test

,textID,sentiment,gender,email_verified,blue_tick,normalized_message_length,normalized_num_messages_sent,normalized_follower_count,normalized_following_count,normalized_seniority,...,week,weekend,well,were,when,whi,who,wish,would,yeah
0,b8f4e560fa,1,1,0,0,0.361702,0.244898,0.897959,0.285714,0.795728,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.5
1,f81a1511b2,1,0,1,1,0.843972,0.979592,0.367347,0.857143,0.955315,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
2,3e9e3f0d69,1,1,1,1,0.297872,0.612245,0.306122,0.489796,0.946231,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,a068b95bd9,-1,0,0,0,0.397163,0.102041,0.020408,0.081633,0.749816,...,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0
4,e6da2d1835,-1,0,0,0,0.751773,0.163265,0.040816,0.122449,0.914314,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12021,d44a6cf5cd,1,1,1,1,0.595745,0.714286,0.469388,0.653061,0.946722,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
12022,35ee036565,1,1,0,0,0.950355,0.448980,0.489796,0.244898,0.736558,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
12023,10c23a0f46,1,1,1,1,0.687943,0.469388,0.775510,0.306122,0.954088,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
12024,a2dd416423,-1,1,1,0,0.411348,0.061224,0.122449,0.102041,0.872821,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## Feature Selection Function

In [146]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2


def feature_selection(df, k=20):
    # Extract the target variable 'sentiment'
    Y = df['sentiment']

    # Drop the 'sentiment' column
    X_cat = df.drop(columns=['sentiment'])

    # Save 'textID' column for later
    textID_column = X_cat['textID']

    # Drop 'textID' and 'sentiment' columns from the feature DataFrame
    X_cat = X_cat.drop(columns=['textID'])

    # Initialize SelectKBest with chi-squared as the scoring function and k=k
    chi2_features = SelectKBest(chi2, k=k)

    # Fit SelectKBest to the data and transform it
    X_cat_kbest = chi2_features.fit_transform(X_cat, Y)

    # Get the selected features
    selected_features = X_cat.columns[chi2_features.get_support()]

    # Create a DataFrame with selected features
    selected_features_df = pd.DataFrame(X_cat_kbest, columns=selected_features)

    # Add 'textID' column back to the selected features DataFrame
    selected_features_df['textID'] = df['textID']
    selected_features_df['sentiment'] = df['sentiment']

    return selected_features_df


## K Fold

In [147]:
# from sklearn.model_selection import KFold
# from sklearn.tree import DecisionTreeClassifier, plot_tree
# from sklearn.metrics import roc_auc_score, accuracy_score
# 
# # Define your dataset (X, y)
# X = df.drop(columns=['sentiment'])
# y = df['sentiment']
# 
# # Define the number of folds for cross-validation
# k = 10
# 
# # Initialize KFold with the number of folds
# kf = KFold(n_splits=k, shuffle=True, random_state=42)
# 
# res = []
# # Perform k-fold cross-validation
# for fold, (train_index, test_index) in enumerate(kf.split(X)):
#     print(f"Fold {fold + 1}:")
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
# 
#     train_df = pd.concat([X_train, y_train], axis=1)
#     test_df = pd.concat([X_test, y_test], axis=1)
# 
#     # Step 1: Preprocess the data
#     train_processed = preprocess_data(train_df)
# 
#     # Step 2: Extract features
#     train_features, new_columns_train, df_output_train = extract_features(train_processed)
# 
#     # Step 3: Perform feature representation
#     train_represented = feature_representation(train_features, new_columns_train, df_output_train)
# 
#     # Step 4: Perform feature selection
#     train_selected = feature_selection(train_represented)
# 
#     x_train_selected = train_selected.drop(columns=['sentiment', 'textID'])
#     y_train_selected = train_selected['sentiment']
# 
#     # define model
#     model = DecisionTreeClassifier(criterion='entropy', random_state=42)
# 
#     print(x_train_selected)
#     print(train_index)
# 
#     # train it using the training indexes
#     model.fit(x_train_selected.iloc[train_index], y_train_selected.iloc[train_index])
# 
#     # test it and get accuracy using the validation indexes
#     acc = roc_auc_score(y_train_selected.iloc[test_index], model.predict(x_train_selected.iloc[test_index]))
# 
#     # save results
#     res.append({'fold': fold + 1,
#                 'acc': acc})
# 
# res = pd.DataFrame(res)
# res
# 


## Holdout Data Validation

In [160]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

# Define dataset (X, y)
X = df.drop(columns=['sentiment'])
y = df['sentiment']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Merge X_train and y_train into one DataFrame for feature extraction
train_df = pd.concat([X_train, y_train], axis=1)

# Merge X_test and y_test into one DataFrame for feature extraction
test_df = pd.concat([X_test, y_test], axis=1)

# Step 1: Preprocess the training data
train_processed = preprocess_data(train_df)

# Step 1: Preprocess the testing data
test_processed = preprocess_data(test_df)

# Step 2: Extract features from the training data
train_features,new_columns_train, df_output_train = extract_features(train_processed)

# Step 2: Extract features from the testing data
test_features,new_columns_test, df_output_test= extract_features(test_processed)

# Step 3: Perform feature representation on training data
train_represented = feature_representation(train_features,new_columns_train, df_output_train )

# Step 3: Perform feature representation on testing data
test_represented = feature_representation(test_features, new_columns_test, df_output_test)
# Step 4: Perform feature selection on training data
train_selected = feature_selection(train_represented)

# Prepare selected features for training
x_train_selected = train_selected.drop(columns=['sentiment', 'textID'])
y_train_selected = train_selected['sentiment']

# Prepare all features for testing (without feature selection)
x_test_all = test_represented.drop(columns=['sentiment', 'textID'])
y_test_selected = test_represented['sentiment']
# Train the model
model = DecisionTreeClassifier(criterion='entropy', random_state=42)
model.fit(x_train_selected, y_train_selected)

# Evaluate the model using all features in the testing data
roc_auc = roc_auc_score(y_test_selected, model.predict(x_test_all))
print(f"Test ROC AUC Score: {roc_auc:.2f}")


/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_31221/1051864747.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['email'] = df['email'].fillna('unknown')
/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_31221/1051864747.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embedded_content'] = df['embedded_content'].apply(lambda x: impute_missing_values(x, embedded_content_prob))
/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_31221/1051864747.py:37: SettingWithCop

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- about
- again
- all
- am
- an
- ...


In [153]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import roc_auc_score, accuracy_score

# Define dataset (X, y)
X = df.drop(columns=['sentiment'])
y = df['sentiment']

#Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Save auc-roc results
res = []

# Merge x,y for feature extraction 
train_df = pd.concat([X_train, y_train], axis=1)

# Step 1: Preprocess the data
train_processed = preprocess_data(train_df)

# Step 2: Extract features
train_features, new_columns_train, df_output_train = extract_features(train_processed)

# Step 3: Perform feature representation
train_represented = feature_representation(train_features, new_columns_train, df_output_train)

# Step 4: Perform feature selection
train_selected = feature_selection(train_represented)

x_train_selected = train_selected.drop(columns=['sentiment', 'textID'])
y_train_selected = train_selected['sentiment']

# define model baseline
model = DecisionTreeClassifier(criterion='entropy', random_state=42)
# train it
pred = model.fit(x_train_selected, y_train_selected)
print(f"Train roc_auc_score: {roc_auc_score(y_train_selected, model.predict(x_train_selected)):.2}")

##predinting on the test set to calculate test acuracy
print(f"Test roc_auc_score: {roc_auc_score(y_test, model.predict(X_test)):.2}")


# # Hyperparameter turning (max_depth)
# for max_depth in range(1,11):
#     
#    
#     
#     
#     
#     # test it and get accuracy using the validation indexes
#     acc = roc_auc_score(y_train_selected, model.predict(x_train_selected))
#     
#     # save results
# res.append({'roc-auc': acc})
# 
# res = pd.DataFrame(res)
# res



/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_31221/1051864747.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['email'] = df['email'].fillna('unknown')
/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_31221/1051864747.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embedded_content'] = df['embedded_content'].apply(lambda x: impute_missing_values(x, embedded_content_prob))
/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_31221/1051864747.py:37: SettingWithCop

Train roc_auc_score: 1.0


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- account_creation_date
- date_of_new_follow
- date_of_new_follower
- email
- embedded_content
- ...
Feature names seen at fit time, yet now missing:
- email_ending_edu
- email_ending_ke
- embedded_content_False
- embedded_content_link
- embedded_content_mp4
- ...
